In [1]:
import numpy as np
from keras.models import Sequential 
from keras.layers import Dense, Reshape, Flatten
from time import gmtime, strftime
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model

Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

# Data loading and processing

In [3]:
# onehot encode
onehot_label = ["0","1","2","3","4","5","6","7","8","9","+","-"," "]
encoder = LabelEncoder()
encoder.fit(onehot_label)

LabelEncoder()

In [4]:
# onehot encoder
def one_hotencoder(dataset):
    dataset_onehot = []
    for data in dataset:
        data_onehot = encoder.transform(list(data))
        data_onehot = np_utils.to_categorical(data_onehot, num_classes=len(onehot_label))
        dataset_onehot.append(data_onehot)
    return dataset_onehot

In [24]:
# onehot decoder
def decoder(list):
    label_index = np.argmax(list, axis=1)
    label = encoder.inverse_transform(label_index)
    #return label
    return "".join(i for i in label)

In [6]:
x_train_path = "data/trainData.npy"
y_train_path = "data/trainTarget.npy"

In [7]:
def loadnpy(path):
    data = np.load(path)
    return data

In [8]:
x_train = loadnpy(x_train_path)
y_train = loadnpy(y_train_path)

# (7, 13)
#print(x_train[0].shape)
# (4, 13)
#print(y_train[0].shape)

# Build Model

In [9]:
def bulidModel(x_shape):
    model = Sequential()
    model.add(Dense(units=256, input_shape=(7, 13), activation='relu'))
    model.add(Flatten())
    model.add(Dense(units=64,activation='relu'))
    model.add(Dense(units=32,activation='relu'))
    model.add(Dense(units=16,activation='relu'))
    model.add(Dense(units=52,activation='relu'))
    model.add(Reshape((4, 13)))
    model.summary()
    
    return model


In [ ]:
print("Build Model")

model = bulidModel(x_train.shape)
model.compile(loss='mse', optimizer='adam')#adam

print("Model Training")
model_history = model.fit(x_train , y_train , batch_size=64 , epochs=100 , verbose=1, validation_split=0.2, shuffle=True)

time = strftime("%Y%m%d_%H_%M", gmtime())

model.save('model+'+time+'.h5')

Build Model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 7, 256)            3584      
_________________________________________________________________
flatten_2 (Flatten)          (None, 1792)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                114752    
_________________________________________________________________
dense_8 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_9 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_10 (Dense)             (None, 52)                884       
_________________________________________________________________
reshape_2 (Reshape)          (None, 4, 13)             0        

# Testing

In [ ]:
x_test_path = "data/x_test.npy"
y_test_path = "data/y_test.npy"
#model_path = "model_20190429_14_44.h5"

In [ ]:
x_test = loadnpy(x_test_path)
y_test = loadnpy(y_test_path)
#model = load_model(model_path)

In [ ]:
#output = model.predict(x_test)
output = model.predict_classes(x_test)

In [ ]:
mse = ((y_test - output)**2).mean(axis=None)
print("MSE : ", mse) 
#mse : 0.048510086 for model model_20190429_14_44.h5

In [ ]:
count = 0
for i in range(len(y_test)):
    q = decoder(x_test[i])
    guess = decoder(output[i])
    correct = decoder(y_test[i])
    print('Q', q, end=' ')
    print('T', correct, end=' ')
    #print("\n", correct, guess)
    if correct == guess:
        print('☑', end=' ')
        count += 1
    else:
        print('☒', end=' ')
    print(guess)

print("Precision : ", count/1000)